In [1]:
from tensorflow import keras
keras.__version__

'2.5.0'

In [2]:
import tensorflow as tf
tf.__version__

'2.5.0'

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [4]:
tf.debugging.set_log_device_placement(True)

try:
  # 유효하지 않은 GPU 장치를 명시
  with tf.device('/device:GPU:1'):
    a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
    b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
    c = tf.matmul(a, b)
except RuntimeError as e:
  print(e)

Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:1


In [5]:
import tensorflow as tf

tf.compat.v1.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [6]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array, save_img

# 변환하려는 이미지 경로
target_image_path = './datasets/portrait.png'
# 스타일 이미지 경로
style_reference_image_path = './datasets/S_portrait.jpg'

# 생성된 사진의 차원
width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width * img_height / height)

In [7]:
import numpy as np
from tensorflow.keras.applications import vgg19

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    # ImageNet의 평균 픽셀 값을 더함
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [8]:
from tensorflow.keras import backend as K

target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))

# 생성된 이미지를 담을 플레이스홀더
combination_image = K.placeholder((1, img_height, img_width, 3))

# 세 개의 이미지를 하나의 배치로 합침
input_tensor = K.concatenate([target_image,
                              style_reference_image,
                              combination_image], axis=0)

# 세 이미지의 배치를 입력으로 받는 VGG 네트워크를 만듦
# 이 모델은 사전 훈련된 ImageNet 가중치를 로드
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet',
                    include_top=False)
print('모델 로드 완료.')

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA TITAN RTX, pci bus id: 0000:67:00.0, compute capability: 7.5
/job:localhost/replica:0/task:0/device:GPU:1 -> device: 1, name: NVIDIA TITAN RTX, pci bus id: 0000:68:00.0, compute capability: 7.5

모델 로드 완료.


In [9]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

In [10]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram


def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

In [11]:
def total_variation_loss(x):
    a = K.square(
        x[:, :img_height - 1, :img_width - 1, :] - x[:, 1:, :img_width - 1, :])
    b = K.square(
        x[:, :img_height - 1, :img_width - 1, :] - x[:, :img_height - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [12]:
# 층 이름과 활성화 텐서를 매핑한 딕셔너리
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
# 콘텐츠 손실에 사용할 층
content_layer = 'block5_conv2'
# 스타일 손실에 사용할 층
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']
# 손실 항목의 가중치 평균에 사용할 가중치
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

# 모든 손실 요소를 더해 하나의 스칼라 변수로 손실을 정의
loss = K.variable(0.)
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(target_image_features,
                                      combination_features)
for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(style_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [13]:
# 손실에 대한 생성된 이미지의 그래디언트를 구함
grads = K.gradients(loss, combination_image)[0]

# 현재 손실과 그래디언트의 값을 추출하는 케라스 Function 객체
fetch_loss_and_grads = K.function([combination_image], [loss, grads])


class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()

In [14]:
from scipy.optimize import fmin_l_bfgs_b
import time

result_prefix = 'style_transfer_result'
iterations = 10000

# 뉴럴 스타일 트랜스퍼의 손실을 최소화하기 위해 생성된 이미지에 대해 L-BFGS 최적화를 수행
# 초기 값은 타깃 이미지
# scipy.optimize.fmin_l_bfgs_b 함수가 벡터만 처리할 수 있기 때문에 이미지를 펼침
x = preprocess_image(target_image_path)
x = x.flatten()
for i in range(iterations):
    print('반복 횟수:', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x,
                                     fprime=evaluator.grads, maxfun=20)
    print('현재 손실 값:', min_val)
    # 생성된 현재 이미지를 저장
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    fname = result_prefix + '_at_iteration_%d.png' % i
    save_img(fname, img)
    end_time = time.time()
    print('저장 이미지: ', fname)
    print('%d 번째 반복 완료: %ds' % (i, end_time - start_time))

반복 횟수: 0
현재 손실 값: 1348070400.0
저장 이미지:  style_transfer_result_at_iteration_0.png
0 번째 반복 완료: 12s
반복 횟수: 1
현재 손실 값: 772688260.0
저장 이미지:  style_transfer_result_at_iteration_1.png
1 번째 반복 완료: 3s
반복 횟수: 2
현재 손실 값: 566427000.0
저장 이미지:  style_transfer_result_at_iteration_2.png
2 번째 반복 완료: 3s
반복 횟수: 3
현재 손실 값: 466228400.0
저장 이미지:  style_transfer_result_at_iteration_3.png
3 번째 반복 완료: 4s
반복 횟수: 4
현재 손실 값: 408990900.0
저장 이미지:  style_transfer_result_at_iteration_4.png
4 번째 반복 완료: 4s
반복 횟수: 5
현재 손실 값: 359317150.0
저장 이미지:  style_transfer_result_at_iteration_5.png
5 번째 반복 완료: 3s
반복 횟수: 6
현재 손실 값: 321312740.0
저장 이미지:  style_transfer_result_at_iteration_6.png
6 번째 반복 완료: 3s
반복 횟수: 7
현재 손실 값: 290569730.0
저장 이미지:  style_transfer_result_at_iteration_7.png
7 번째 반복 완료: 4s
반복 횟수: 8
현재 손실 값: 267890220.0
저장 이미지:  style_transfer_result_at_iteration_8.png
8 번째 반복 완료: 4s
반복 횟수: 9
현재 손실 값: 251435460.0
저장 이미지:  style_transfer_result_at_iteration_9.png
9 번째 반복 완료: 4s
반복 횟수: 10
현재 손실 값: 224835330.0
저장 이미지:  style_tr

현재 손실 값: 31028944.0
저장 이미지:  style_transfer_result_at_iteration_85.png
85 번째 반복 완료: 4s
반복 횟수: 86
현재 손실 값: 30599410.0
저장 이미지:  style_transfer_result_at_iteration_86.png
86 번째 반복 완료: 4s
반복 횟수: 87
현재 손실 값: 30138026.0
저장 이미지:  style_transfer_result_at_iteration_87.png
87 번째 반복 완료: 3s
반복 횟수: 88
현재 손실 값: 29668168.0
저장 이미지:  style_transfer_result_at_iteration_88.png
88 번째 반복 완료: 4s
반복 횟수: 89
현재 손실 값: 29277258.0
저장 이미지:  style_transfer_result_at_iteration_89.png
89 번째 반복 완료: 4s
반복 횟수: 90
현재 손실 값: 28963954.0
저장 이미지:  style_transfer_result_at_iteration_90.png
90 번째 반복 완료: 4s
반복 횟수: 91
현재 손실 값: 28681308.0
저장 이미지:  style_transfer_result_at_iteration_91.png
91 번째 반복 완료: 3s
반복 횟수: 92
현재 손실 값: 28387980.0
저장 이미지:  style_transfer_result_at_iteration_92.png
92 번째 반복 완료: 3s
반복 횟수: 93
현재 손실 값: 28197166.0
저장 이미지:  style_transfer_result_at_iteration_93.png
93 번째 반복 완료: 3s
반복 횟수: 94
현재 손실 값: 27951504.0
저장 이미지:  style_transfer_result_at_iteration_94.png
94 번째 반복 완료: 4s
반복 횟수: 95
현재 손실 값: 27744576.0
저장 이미지:  s

현재 손실 값: 17492716.0
저장 이미지:  style_transfer_result_at_iteration_168.png
168 번째 반복 완료: 5s
반복 횟수: 169
현재 손실 값: 17440614.0
저장 이미지:  style_transfer_result_at_iteration_169.png
169 번째 반복 완료: 3s
반복 횟수: 170
현재 손실 값: 17387690.0
저장 이미지:  style_transfer_result_at_iteration_170.png
170 번째 반복 완료: 3s
반복 횟수: 171
현재 손실 값: 17337328.0
저장 이미지:  style_transfer_result_at_iteration_171.png
171 번째 반복 완료: 4s
반복 횟수: 172
현재 손실 값: 17282096.0
저장 이미지:  style_transfer_result_at_iteration_172.png
172 번째 반복 완료: 4s
반복 횟수: 173
현재 손실 값: 17182722.0
저장 이미지:  style_transfer_result_at_iteration_173.png
173 번째 반복 완료: 4s
반복 횟수: 174
현재 손실 값: 17071536.0
저장 이미지:  style_transfer_result_at_iteration_174.png
174 번째 반복 완료: 3s
반복 횟수: 175
현재 손실 값: 17024142.0
저장 이미지:  style_transfer_result_at_iteration_175.png
175 번째 반복 완료: 3s
반복 횟수: 176
현재 손실 값: 16973514.0
저장 이미지:  style_transfer_result_at_iteration_176.png
176 번째 반복 완료: 4s
반복 횟수: 177
현재 손실 값: 16917486.0
저장 이미지:  style_transfer_result_at_iteration_177.png
177 번째 반복 완료: 4s
반복 횟수: 178


현재 손실 값: 13879550.0
저장 이미지:  style_transfer_result_at_iteration_250.png
250 번째 반복 완료: 4s
반복 횟수: 251
현재 손실 값: 13855381.0
저장 이미지:  style_transfer_result_at_iteration_251.png
251 번째 반복 완료: 3s
반복 횟수: 252
현재 손실 값: 13832617.0
저장 이미지:  style_transfer_result_at_iteration_252.png
252 번째 반복 완료: 3s
반복 횟수: 253
현재 손실 값: 13807694.0
저장 이미지:  style_transfer_result_at_iteration_253.png
253 번째 반복 완료: 3s
반복 횟수: 254
현재 손실 값: 13783856.0
저장 이미지:  style_transfer_result_at_iteration_254.png
254 번째 반복 완료: 4s
반복 횟수: 255
현재 손실 값: 13759750.0
저장 이미지:  style_transfer_result_at_iteration_255.png
255 번째 반복 완료: 4s
반복 횟수: 256
현재 손실 값: 13736211.0
저장 이미지:  style_transfer_result_at_iteration_256.png
256 번째 반복 완료: 3s
반복 횟수: 257
현재 손실 값: 13716255.0
저장 이미지:  style_transfer_result_at_iteration_257.png
257 번째 반복 완료: 3s
반복 횟수: 258
현재 손실 값: 13695903.0
저장 이미지:  style_transfer_result_at_iteration_258.png
258 번째 반복 완료: 4s
반복 횟수: 259
현재 손실 값: 13672142.0
저장 이미지:  style_transfer_result_at_iteration_259.png
259 번째 반복 완료: 4s
반복 횟수: 260


현재 손실 값: 12153146.0
저장 이미지:  style_transfer_result_at_iteration_332.png
332 번째 반복 완료: 3s
반복 횟수: 333
현재 손실 값: 12133074.0
저장 이미지:  style_transfer_result_at_iteration_333.png
333 번째 반복 완료: 3s
반복 횟수: 334
현재 손실 값: 12110982.0
저장 이미지:  style_transfer_result_at_iteration_334.png
334 번째 반복 완료: 4s
반복 횟수: 335
현재 손실 값: 12097449.0
저장 이미지:  style_transfer_result_at_iteration_335.png
335 번째 반복 완료: 3s
반복 횟수: 336
현재 손실 값: 12084477.0
저장 이미지:  style_transfer_result_at_iteration_336.png
336 번째 반복 완료: 3s
반복 횟수: 337
현재 손실 값: 12069021.0
저장 이미지:  style_transfer_result_at_iteration_337.png
337 번째 반복 완료: 3s
반복 횟수: 338
현재 손실 값: 12053266.0
저장 이미지:  style_transfer_result_at_iteration_338.png
338 번째 반복 완료: 4s
반복 횟수: 339
현재 손실 값: 12041182.0
저장 이미지:  style_transfer_result_at_iteration_339.png
339 번째 반복 완료: 4s
반복 횟수: 340
현재 손실 값: 12029492.0
저장 이미지:  style_transfer_result_at_iteration_340.png
340 번째 반복 완료: 3s
반복 횟수: 341
현재 손실 값: 12019654.0
저장 이미지:  style_transfer_result_at_iteration_341.png
341 번째 반복 완료: 4s
반복 횟수: 342


...

현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9660.png
9660 번째 반복 완료: 3s
반복 횟수: 9661
현재 손실 값: 8139426.5
저장 이미지:  style_transfer_result_at_iteration_9661.png
9661 번째 반복 완료: 3s
반복 횟수: 9662
현재 손실 값: 8139426.5
저장 이미지:  style_transfer_result_at_iteration_9662.png
9662 번째 반복 완료: 2s
반복 횟수: 9663
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9663.png
9663 번째 반복 완료: 3s
반복 횟수: 9664
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9664.png
9664 번째 반복 완료: 2s
반복 횟수: 9665
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9665.png
9665 번째 반복 완료: 3s
반복 횟수: 9666
현재 손실 값: 8139426.5
저장 이미지:  style_transfer_result_at_iteration_9666.png
9666 번째 반복 완료: 2s
반복 횟수: 9667
현재 손실 값: 8139426.5
저장 이미지:  style_transfer_result_at_iteration_9667.png
9667 번째 반복 완료: 2s
반복 횟수: 9668
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9668.png
9668 번째 반복 완료: 2s
반복 횟수: 9669
현재 손실 값: 8139426.5
저장 이미지:  style_transfer_result_at_iteration_9669.png
9669 번째 반복

현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9741.png
9741 번째 반복 완료: 3s
반복 횟수: 9742
현재 손실 값: 8139426.5
저장 이미지:  style_transfer_result_at_iteration_9742.png
9742 번째 반복 완료: 3s
반복 횟수: 9743
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9743.png
9743 번째 반복 완료: 3s
반복 횟수: 9744
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9744.png
9744 번째 반복 완료: 2s
반복 횟수: 9745
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9745.png
9745 번째 반복 완료: 2s
반복 횟수: 9746
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9746.png
9746 번째 반복 완료: 3s
반복 횟수: 9747
현재 손실 값: 8139426.5
저장 이미지:  style_transfer_result_at_iteration_9747.png
9747 번째 반복 완료: 2s
반복 횟수: 9748
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9748.png
9748 번째 반복 완료: 2s
반복 횟수: 9749
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9749.png
9749 번째 반복 완료: 3s
반복 횟수: 9750
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9750.png
9750 번째 반복

현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9822.png
9822 번째 반복 완료: 2s
반복 횟수: 9823
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9823.png
9823 번째 반복 완료: 2s
반복 횟수: 9824
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9824.png
9824 번째 반복 완료: 2s
반복 횟수: 9825
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9825.png
9825 번째 반복 완료: 2s
반복 횟수: 9826
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9826.png
9826 번째 반복 완료: 3s
반복 횟수: 9827
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9827.png
9827 번째 반복 완료: 3s
반복 횟수: 9828
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9828.png
9828 번째 반복 완료: 2s
반복 횟수: 9829
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9829.png
9829 번째 반복 완료: 2s
반복 횟수: 9830
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9830.png
9830 번째 반복 완료: 3s
반복 횟수: 9831
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9831.png
9831 번째 반복

현재 손실 값: 8139426.5
저장 이미지:  style_transfer_result_at_iteration_9903.png
9903 번째 반복 완료: 2s
반복 횟수: 9904
현재 손실 값: 8139426.5
저장 이미지:  style_transfer_result_at_iteration_9904.png
9904 번째 반복 완료: 3s
반복 횟수: 9905
현재 손실 값: 8139426.5
저장 이미지:  style_transfer_result_at_iteration_9905.png
9905 번째 반복 완료: 2s
반복 횟수: 9906
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9906.png
9906 번째 반복 완료: 2s
반복 횟수: 9907
현재 손실 값: 8139426.5
저장 이미지:  style_transfer_result_at_iteration_9907.png
9907 번째 반복 완료: 3s
반복 횟수: 9908
현재 손실 값: 8139426.5
저장 이미지:  style_transfer_result_at_iteration_9908.png
9908 번째 반복 완료: 2s
반복 횟수: 9909
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9909.png
9909 번째 반복 완료: 2s
반복 횟수: 9910
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9910.png
9910 번째 반복 완료: 3s
반복 횟수: 9911
현재 손실 값: 8139426.5
저장 이미지:  style_transfer_result_at_iteration_9911.png
9911 번째 반복 완료: 2s
반복 횟수: 9912
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9912.png
9912 번째 반복

현재 손실 값: 8139426.5
저장 이미지:  style_transfer_result_at_iteration_9984.png
9984 번째 반복 완료: 2s
반복 횟수: 9985
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9985.png
9985 번째 반복 완료: 2s
반복 횟수: 9986
현재 손실 값: 8139426.5
저장 이미지:  style_transfer_result_at_iteration_9986.png
9986 번째 반복 완료: 3s
반복 횟수: 9987
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9987.png
9987 번째 반복 완료: 3s
반복 횟수: 9988
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9988.png
9988 번째 반복 완료: 3s
반복 횟수: 9989
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9989.png
9989 번째 반복 완료: 2s
반복 횟수: 9990
현재 손실 값: 8139426.5
저장 이미지:  style_transfer_result_at_iteration_9990.png
9990 번째 반복 완료: 2s
반복 횟수: 9991
현재 손실 값: 8139426.5
저장 이미지:  style_transfer_result_at_iteration_9991.png
9991 번째 반복 완료: 3s
반복 횟수: 9992
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9992.png
9992 번째 반복 완료: 2s
반복 횟수: 9993
현재 손실 값: 8139426.0
저장 이미지:  style_transfer_result_at_iteration_9993.png
9993 번째 반복